In [ ]:
import os, glob
import torch
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf
import torchvision
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.utils.data import Subset
from torch.amp import autocast, GradScaler
from sklearn.metrics import classification_report
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive (7).zip to archive (7).zip


In [ ]:
!mkdir -p "/content/drive/MyDrive/datasets/emotion/raw"
!mv "/content/archive (7).zip" "/content/drive/MyDrive/datasets/emotion/"
!unzip -q "/content/drive/MyDrive/datasets/emotion/archive (7).zip" -d "/content/drive/MyDrive/datasets/emotion/raw"
!ls "/content/drive/MyDrive/datasets/emotion/raw"

replace /content/drive/MyDrive/datasets/emotion/raw/test/angry/im0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: test  train


In [ ]:
data_dir = "/content/drive/MyDrive/datasets/emotion/raw"
train_dir = os.path.join(data_dir, "train")
test_dir  = os.path.join(data_dir, "test")

In [ ]:
print("Train classes:", os.listdir(train_dir))
for c in sorted(os.listdir(train_dir)):
    n = len(glob.glob(os.path.join(train_dir, c, "*")))
    print(" ", c, n)

print("\nTest classes:", os.listdir(test_dir))
for c in sorted(os.listdir(test_dir)):
    n = len(glob.glob(os.path.join(test_dir, c, "*")))
    print(" ", c, n)

Train classes: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
  angry 3995
  disgusted 436
  fearful 4097
  happy 7215
  neutral 4965
  sad 4830
  surprised 2309

Test classes: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
  angry 958
  disgusted 111
  fearful 1024
  happy 1774
  neutral 1233
  sad 1247
  surprised 831


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(12),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(train_dir, transform=test_transform)
test_dataset  = datasets.ImageFolder(test_dir,  transform=test_transform)

In [ ]:
# reproducible split
n = len(train_dataset)
g = torch.Generator().manual_seed(42)
idx = torch.randperm(n, generator=g).tolist()

val_size = int(0.15 * n)
val_idx = idx[:val_size]
trn_idx = idx[val_size:]

train_subset = Subset(train_dataset, trn_idx)
val_subset = Subset(val_dataset,   val_idx)

In [ ]:
# sampler weights for TRAIN subset only
targets = [train_dataset.samples[i][1] for i in trn_idx]
class_counts = Counter(targets)
class_weights = {c: 1.0 / class_counts[c] for c in class_counts}
sample_weights = torch.tensor([class_weights[t] for t in targets], dtype=torch.float)

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_subset, batch_size=64, sampler=sampler, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_subset,   batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
# 1) get labels for every training sample
targets = [y for _, y in train_dataset.samples]

# 2) count how many per class
class_counts = Counter(targets)

# 3) weight per class = inverse of count
class_weights = {c: 1.0 / class_counts[c] for c in class_counts}

# 4) weight per sample = weight of its class
sample_weights = torch.tensor([class_weights[t] for t in targets], dtype=torch.double)

# 5) sampler will oversample rare classes
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# IMPORTANT: use sampler (do NOT use shuffle=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    sampler=sampler,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

class_names = train_dataset.classes
num_classes = len(class_names)
print("Classes:", class_names)
print("Train images:", len(train_dataset))
print("Test images:", len(test_dataset))

Classes: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Train images: 27847
Test images: 7178


In [ ]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

for p in model.parameters():
    p.requires_grad = True

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 156MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(train_loader),
    epochs=15
)

In [ ]:
scaler = GradScaler(enabled=(device.type == "cuda"))

def train_one_epoch_amp(model, loader, optimizer, criterion, device, scheduler=None):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast(device_type=device.type, enabled=(device.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if scheduler is not None:
            scheduler.step()

        total_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # optional: autocast here for faster eval on GPU
        with autocast(device_type=device.type, enabled=(device.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)

        total_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total



In [ ]:
epochs = 8
best_acc = 0.0
best_state = None

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch_amp(model, train_loader, optimizer, criterion, device, scheduler)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    if val_acc > best_acc:
        best_acc = val_acc
        best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train: {train_loss:.4f}, {train_acc:.4f} | "
          f"Val:   {val_loss:.4f}, {val_acc:.4f} | "
          f"BestVal: {best_acc:.4f}")

if best_state is not None:
    model.load_state_dict(best_state)

# FINAL test (once)
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Final Test: {test_loss:.4f}, {test_acc:.4f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
all_preds, all_labels = [], []

model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        pred = model(x).argmax(1).cpu()
        all_preds += pred.tolist()
        all_labels += y.tolist()

print(classification_report(all_labels, all_preds, target_names=class_names))


NameError: name 'model' is not defined

In [ ]:
counts = Counter([y for _, y in train_dataset.samples])
for i, name in enumerate(class_names):
    print(name, counts[i])


NameError: name 'Counter' is not defined

In [ ]:
def predict_image(img_path, model, transform, class_names):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(x)
        _, pred = torch.max(outputs, 1)

    print("Predicted emotion:", class_names[pred.item()])
    display(img.resize((224, 224)))

# Example: change the path to any image you like
sample_path = "/content/emotions/test/disgusted/im109.png"
predict_image(sample_path, model, test_transform, class_names)


In [ ]:
torch.save(model.state_dict(), "emotion_efficientnet_b0.pth")


In [ ]:
from google.colab import files
files.download("emotion_efficientnet_b0.pth")

In [ ]:
import json
with open("class_names.json","w") as f:
    json.dump(train_dataset.classes, f)
files.download("class_names.json")
